In [5]:
#General modules
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt

# Text processing modules
import string
import re
import nltk
nltk.download('punkt')

# ML modules
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Preprocessing functions
def text_preprocessing(text, stop_words):
    """
    Performs tokenization and simple preprocessing.
    Args : 
        @param text (String): Text that needs preprocessing
        @return: text (String) : Preprocessed text
    """
    bad_symbols = re.compile('[/(){}\[\]\|@,;+_#]')
    fine_symbols = re.compile('[^0-9абвгдѓежзѕијклљмнњопрстќуфхцчџш ]')
    stopwords_set = stop_words
    lat_to_cyr = {'kj' : 'ќ', 'gj' : 'ѓ', 'zh' : 'ж', 'ch' : 'ч', 'sh' : 'ш', 'dj' : 'ѓ',
              'a' : 'а', 'b' : 'б', 'c' : 'ц', 'd' : 'д', 'e' : 'е', 'f' : 'ф', 'g' : 'г',
              'h' : 'х', 'i' : 'и', 'j' : 'ј', 'k' : 'к', 'l' : 'л', 'm' : 'м', 'n' : 'н', 
              'o' : 'о', 'p' : 'п', 'q' : 'љ', 'r' : 'р', 's' : 'с', 't' : 'т', 'u' : 'у',
              'v' : 'в', 'w' : 'њ', 'x' : 'џ', 'y' : 'ѕ',  'z' : 'з'
             }
    
    text = text.lower()
    
    for item, value in lat_to_cyr.items():
        text = re.sub(item, value, text)
    
    text = bad_symbols.sub(' ', text)

    text = fine_symbols.sub('', text)
    
    text = ' '.join([x for x in text.split() if x and x not in stopwords_set])

    return text.strip()



# Load the dataset
def load_dataset(source):
    # Loading the data from the source
    data = pd.read_csv(source)

    # Spliting into questions, answers and categories
    questions = data.iloc[:, 0]
    answers = data.iloc[:, 1]
    categories = data.iloc[:, 2]
    category_codes = data.iloc[:, 3]
    
    return questions, answers, categories, category_codes


# Load the stop words
def load_stop_words(source):
    file = open(source, 'r')
    sw=file.read().split()
    file.close()
    return sw

[nltk_data] Downloading package punkt to /home/filip/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Loading the database of questions and building the corpus
qus, ans, cat, cat_codes = load_dataset('dataset_brainster.csv')
sw = load_stop_words('stop_words.txt')

corpus = [text_preprocessing(q, sw) for q in qus]

In [7]:
qus

0      Колку{време}трае академијата(за)дигитален марк...
1      колку месеци недели е академијата за дигитален...
2      Како се одвива наставата на академијата за диг...
3                   Дали наставата за маркетинг е online
4        Колку часа неделно има на маркетинг академијата
                             ...                        
518     Дали добивам диплома по завршување на Академија?
519           Како да аплицирам на академијата за UX/UI?
520    Како ги одбирате студентите на академијата за ...
521    Дали може да се приклучам на листа на кандидат...
522    Кои се придобивките од посета на Академијата з...
Name: questions, Length: 523, dtype: object

In [8]:
# Builting the Count-Vectorized features
count_vectorizer = CountVectorizer(strip_accents='unicode') #probaj bez max_features
count_vectorizer.fit(corpus)
corpus_vec = count_vectorizer.transform(corpus)

corpus_vec_df = pd.DataFrame(data = corpus_vec.todense(), columns = count_vectorizer.get_feature_names())

In [9]:
corpus_vec_df.head()

,15,1630,24,автоматско,адс,академии,академиите,академија,академијата,акредитиран,...,чини,школувањето,јава,јазик,јазици,јак,јљуерѕ,ља,њебдривер,њебсите
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Building the TF-IDF Vectorized Features
tfidf_vectorizer = TfidfVectorizer(strip_accents='unicode')
tfidf_vectorizer.fit(corpus)
corpus_tfidif = tfidf_vectorizer.transform(corpus)

corpus_tfidif_df = pd.DataFrame(data = corpus_tfidif.todense(), columns = tfidf_vectorizer.get_feature_names())

In [11]:
corpus_tfidif_df.shape

(523, 499)

## Testing CountVectorizer

In [12]:
# Defining Cosine Similarity Functiom
def cos_sim(new_input, corpus): 
    cosine_function = lambda a, b : np.inner(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    distances = []
    for question in corpus:
        distances.append(cosine_function(new_input.toarray(), question.toarray()))
    max_sim = max(max(max(distances)))
    max_arg = np.argmax(distances)
    return max_arg, max_sim, distances

In [13]:
# Defining Pearson Similarity Function
def pearson_sim(new_input, corpus): 
    distances = []
    for question in corpus:
        distances.append(np.corrcoef(new_input.todense(), question.todense())[0, 1])
    max_sim = max(distances)
    max_arg = np.argmax(distances)
    return max_arg, max_sim, distances

In [14]:
# Defining Dice Similarity Function
def dice_sim(new_input, corpus):
    new_input = new_input.toarray().flatten().astype(bool)
    distances = []
    for question in corpus:
        question = question.toarray().flatten().astype(bool)
        distances.append(2.0*np.logical_and(new_input, question).sum()/(new_input.sum() + question.sum()))
    max_sim = max(distances)
    max_arg = np.argmax(distances)
    return max_arg, max_sim, distances

In [15]:
tq = 'се прифаќа ли ваучер од владата'
tq = text_preprocessing(tq, sw)
tq = count_vectorizer.transform([tq])

In [16]:
# Getting the closest cosine-score and responding
max_arg, max_sim, distances = cos_sim(tq, corpus_vec)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

376
0.4999999999999999
Најблиску до прашањето: како да аплицирам за ваучер
Одговор:  Најпрво избери ја обуката за која сакаш да го искористиш ваучерот, резервирај го твоето место на истата, а нашиот тим ќе ти помогне во целиот процес на апликација за ваучерот. Дополнително може да се информираш на следниот лник https://vauceri.brainster.co/upatstvo/


In [17]:
# Getting the closest pearson-score and responding
max_arg, max_sim, distances = pearson_sim(tq, corpus_vec)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

376
0.49798792756539395
Најблиску до прашањето: како да аплицирам за ваучер
Одговор:  Најпрво избери ја обуката за која сакаш да го искористиш ваучерот, резервирај го твоето место на истата, а нашиот тим ќе ти помогне во целиот процес на апликација за ваучерот. Дополнително може да се информираш на следниот лник https://vauceri.brainster.co/upatstvo/


In [18]:
# Getting the closest dice-score and responding
max_arg, max_sim, distances = dice_sim(tq, corpus_vec)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

376
0.5
Најблиску до прашањето: како да аплицирам за ваучер
Одговор:  Најпрво избери ја обуката за која сакаш да го искористиш ваучерот, резервирај го твоето место на истата, а нашиот тим ќе ти помогне во целиот процес на апликација за ваучерот. Дополнително може да се информираш на следниот лник https://vauceri.brainster.co/upatstvo/


## Testing TF-IDF Vectorizer

In [19]:
tq = 'се прифаќа ли ваучер од владата'
tq = text_preprocessing(tq, sw)
tq = tfidf_vectorizer.transform([tq])

In [20]:
# Getting the closest cosine-score and responding
max_arg, max_sim, distances = cos_sim(tq, corpus_tfidif)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

376
0.6176566731622903
Најблиску до прашањето: како да аплицирам за ваучер
Одговор:  Најпрво избери ја обуката за која сакаш да го искористиш ваучерот, резервирај го твоето место на истата, а нашиот тим ќе ти помогне во целиот процес на апликација за ваучерот. Дополнително може да се информираш на следниот лник https://vauceri.brainster.co/upatstvo/


In [21]:
# Getting the closest pearson-score and responding
max_arg, max_sim, distances = pearson_sim(tq, corpus_tfidif)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

376
0.6161442349675342
Најблиску до прашањето: како да аплицирам за ваучер
Одговор:  Најпрво избери ја обуката за која сакаш да го искористиш ваучерот, резервирај го твоето место на истата, а нашиот тим ќе ти помогне во целиот процес на апликација за ваучерот. Дополнително може да се информираш на следниот лник https://vauceri.brainster.co/upatstvo/


In [22]:
# Getting the closest dice-score and responding
max_arg, max_sim, distances = dice_sim(tq, corpus_tfidif)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

376
0.5
Најблиску до прашањето: како да аплицирам за ваучер
Одговор:  Најпрво избери ја обуката за која сакаш да го искористиш ваучерот, резервирај го твоето место на истата, а нашиот тим ќе ти помогне во целиот процес на апликација за ваучерот. Дополнително може да се информираш на следниот лник https://vauceri.brainster.co/upatstvo/


## Building the WordEmbeddings

In [3]:
# Loading modules
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

#Building the embedding corpus
corpus_embedding = [question.split() for question in corpus]

corpus_embedding[:2]

In [23]:
word_to_int = Tokenizer(
    num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n„“', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0)


text_int = word_to_int.fit_on_texts(corpus)

text_int = word_to_int.texts_to_sequences(corpus)
text_int[0]

[36, 42, 1, 12, 11]

In [26]:
corpus[0]

'време трае академијата дигитален маркетинг'

In [21]:
max(max(text_int))

523

In [27]:
from bpemb import BPEmb
bpemb_mk = BPEmb(lang="mk", dim=300)

In [28]:
embed_corpus = []
for question in corpus:
    embed_corpus.append(bpemb_mk.embed(question).mean(axis = 0))

In [38]:
embed_corpus[1]

array([-0.36950657,  0.18893042,  0.01634129, -0.00867986,  0.11965057,
       -0.25862342,  0.105135  , -0.13501443, -0.11052115, -0.13546571,
        0.14883716, -0.0751087 ,  0.13449328,  0.01924443, -0.17472914,
       -0.03395014, -0.06081856, -0.08908772, -0.02118914, -0.27493757,
        0.2610376 ,  0.14679416,  0.13678773, -0.042043  , -0.06447043,
       -0.03435528, -0.03392672, -0.09542271,  0.00625943,  0.10281787,
        0.01958071,  0.03375972, -0.00518758,  0.04028443, -0.21520887,
        0.16210842, -0.13096914,  0.03470386,  0.03595701, -0.042074  ,
        0.00493985,  0.03202243, -0.12084858, -0.01632114,  0.246698  ,
       -0.08829314, -0.03844229, -0.16857372,  0.09927315, -0.19717327,
        0.03333686, -0.12733844, -0.04869387,  0.04668472,  0.06395758,
       -0.06671644,  0.15618828,  0.25347   ,  0.15353842,  0.09247072,
        0.08692227, -0.03039116, -0.36476973,  0.09982427, -0.11355243,
        0.08249171, -0.01313242, -0.21146175, -0.00904758, -0.14

# НАДОЛУ Е ТЕСТ КОД

In [ ]:
print(qus[0])
print()
print(corpus[112:114])

In [ ]:
(corpus_vec[112].toarray() * corpus_vec[113].toarray()).sum()

In [ ]:
np.inner(corpus_vec[113].toarray()[0], corpus_vec[113].toarray()[0])

In [ ]:
from scipy.spatial.distance import dice, cosine, correlation

print('cosine = ', cosine(tq.toarray(), corpus_vec[89].toarray()))

print('correlation = ', correlation(tq.toarray(), corpus_vec[89].toarray()))

print('dice = ', dice(tq.toarray(), corpus_vec[89].toarray()))

In [ ]:
cosine_function = lambda a, b : np.inner(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
max(max(cosine_function(tq.toarray(), corpus_vec[89].toarray())))

In [ ]:
np.inner(tq.toarray(), corpus_vec[89].toarray())[0,0]

In [ ]:
a = tq
b = corpus_vec[89]
print(a)
print()
print(b)

In [ ]:
#c = a==b
a = tq
b = corpus_vec[89]
def dice_sim(a, b):
    a = a.toarray().flatten().astype(bool)
    b = b.toarray().flatten().astype(bool)
    return 2.0*np.logical_and(a, b).sum()/(a.sum() + b.sum())

dice_sim(a,a)

In [ ]:
dice(a.toarray(), b.toarray())

In [ ]:
dice([0, 1, 0], [0, 1, 0])

In [ ]:
dice_sim(np.asarray([0, 1, 0]), np.asarray([1, 0, 0]))

In [ ]:
p = np.array([0, 1, 0])
q = np.array([0, 1, 0])

print('cosine = ', cosine(p, q))

print('correlation = ', correlation(p, q))

print('dice = ', dice(p, q))

In [ ]:
def dice_sim_2(a, b):
    a = a.astype(bool)
    b = b.astype(bool)
    return 2.0*np.logical_and(a, b).sum()/(a.sum() + b.sum())

dice_sim_2(p, q)

In [ ]:
hasattr(a, ('toarray', 'flatten'))